<a href="https://colab.research.google.com/github/BustamJos3/calculoDiagnosticoRedAireComprimido/blob/BustamJos3%2FFirstContributions/inplementacionCodRAC_potencia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alternative approach: given $D$, find $ΔP$ until $ΔP=0.02*P_{req}$
With that $D$, choose respective accesories and recalculate $D$

[Pressure drop through piping](https://fluids.readthedocs.io/tutorial.html#pressure-drop-through-piping)

In [ ]:
!pip install pint

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from fluids.units import *
from fluids.fittings import *
from fluids.core import Reynolds
from fluids.core import K_from_f
from fluids.core import dP_from_K
from fluids.friction import friction_factor
from fluids.atmosphere import ATMOSPHERE_1976
from fluids.friction import nearest_material_roughness, material_roughness

In [ ]:
atm =ATMOSPHERE_1976(Z=1495)#*u.m properties of air @ 1495[msnm]

In [ ]:
seedD=0.089 #*u.m seed diameter

In [ ]:
import numpy as np

In [ ]:
numberOfMachines=np.array([4,7,7,4,3])
flowPerMachine=np.array([567,85,142,50,40])*1.66667E-5 #*(u.L/u.min) to *(u.m**(3)/u.s) air flow on SI
utilizationCoeff=np.array([1,0.8,0.75,0.6,0.7])
partialQ=numberOfMachines*flowPerMachine*utilizationCoeff
totalQ=np.sum(partialQ) # calculate total flow
totalQ

0.06155845645000001

In [ ]:
workLongitude=20 #*u.m longitude of pipe section
areaFunction=lambda x: (np.pi*x**2)/4 #calculate pipe area
seedA=areaFunction(seedD)
rhoDensity=atm.rho # get rho @ Medellín high
airMu=atm.viscosity(T=298) # dynamic viscosity @ T ambient *u.K
V=totalQ/seedA # compressed air velocity
Re = Reynolds(V=V, D=seedD, rho=rhoDensity, mu=airMu)
epsilon=material_roughness( nearest_material_roughness('compressed air', clean=True), D=seedD, optimism=True )*u.m # roughness for pipe material
fd = friction_factor(Re, eD=epsilon.to_tuple()[0]/seedD) # friction factor
fd

0.02495281749267308

In [ ]:
secondaryPipeD=seedD*0.8
secondaryPipeSections=5

In [ ]:
k=0# klosses for ff and accesories
k+=K_from_f(fd=fd, L=workLongitude, D=seedD)
kAccesories=[('Elbow, 90°, Long-radius (R/D = 1.5), All types',5),('Tee, Run, Screwed', 5), ('Valve, Ball, Reduced trim, Beta = 0.9',1)] #elbows, tees, valves
for i in kAccesories:
    k+=Hooper2K(Di=seedD,Re=Re,name=i[0])*i[1] # seedD.to_tuple()[0]
KContractions=secondaryPipeSections*contraction_sharp(Di1=seedD,Di2=secondaryPipeD,fd=fd,roughness=epsilon) #loss for contraction to SecondarySection
k+=KContractions
k

27.056748363760533

In [ ]:
inletP=6.3*u.bar
dropP=dP_from_K(k,rho=rhoDensity,V=V)# calculate drop preassure
dropP=dropP*u.Pa
dropP.to('bar')/inletP*100 #DeltaP=2%?

0.2225792420415321 <Unit('dimensionless')>

# Plain implementation

In [ ]:
!pip install fluids
!pip install pint

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 24.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.5/269.5 KB 5.5 MB/s eta 0:00:00


In [ ]:
import numpy as np #external modules
from fluids.atmosphere import ATMOSPHERE_1976

## $T_P$

In [ ]:
numberOfMachines=np.array([4,7,7,4,3])
flowPerMachines=np.array([567,85,142,50,40])*1.66667E-5 #*(u.L/u.min) to *(u.m**(3)/u.s) air flow on SI
utilizationCoeffs=np.array([1,0.8,0.75,0.6,0.7])
partialQ=numberOfMachines*flowPerMachines*utilizationCoeffs
Q=np.sum(partialQ) # calculate total flow
securityF=np.array([0.15,0.1,0.1,0.2])
securityFatQ=securityF*Q
totalQ=np.sum(securityFatQ)
atm=ATMOSPHERE_1976(1495)#*u.m properties of air @ 1495[msnm]
seedD=0.089 #*u.m seed diameter
workLongitude=20 #*u.m longitude of pipe section
secondaryPipeDia=seedD*0.8 #diameter to contraction from TP to TS
secondaryPipeSecs=5 #number of TS
kAccesories=[('Elbow, 90°, Long-radius (R/D = 1.5), All types',secondaryPipeSecs),('Tee, Run, Screwed', secondaryPipeSecs), ('Valve, Ball, Reduced trim, Beta = 0.9',1)] #elbows, tees, valves
inP=6.3 #*u.bar

In [ ]:
def checkdP_fromD(seedDia, atmObj, workLong, kAcces, secondPipeD, secondPipeSects, inletP,Qtot):
    import numpy as np #modules
    from fluids.fittings import Hooper2K, contraction_sharp
    from fluids.core import Reynolds
    from fluids.core import K_from_f
    from fluids.core import dP_from_K
    from fluids.friction import friction_factor
    from fluids.friction import nearest_material_roughness, material_roughness
    
    areaFunction=lambda x: (np.pi*x**2)/4 #calculate pipe area
    seedA=areaFunction(seedDia)
    rhoDensity=atmObj.rho # get rho @ Medellín high
    airMu=atmObj.viscosity(298) # dynamic viscosity @ T ambient *u.K
    V=Qtot/seedA # compressed air velocity
    Re = Reynolds(V=V, D=seedDia, rho=rhoDensity, mu=airMu)
    epsilon=material_roughness( nearest_material_roughness('compressed air', clean=True), D=seedDia, optimism=True ) #*u.m roughness for pipe material
    fd = friction_factor(Re, eD=epsilon/seedDia) # friction factor
    k=0# klosses for ff and accesories
    k+=K_from_f(fd=fd, L=workLong, D=seedDia)
    for i in kAcces:
        k+=Hooper2K(Di=seedDia,Re=Re,name=i[0])*i[1] # seedD.to_tuple()[0]
    KContractions=secondPipeSects*contraction_sharp(Di1=seedDia,Di2=secondPipeD,fd=fd,roughness=epsilon) #loss for contraction to SecondarySection
    k+=KContractions
    dropP=dP_from_K(k,rho=rhoDensity,V=V)# calculate drop preassure
    dP=dropP/(inletP*10E5) # DeltaP=2%?
    return dP

In [ ]:
checkdP_fromD(seedDia=seedD, atmObj=atm, workLong=workLongitude, kAcces=kAccesories, secondPipeD=secondaryPipeDia, secondPipeSects=secondaryPipeSecs, inletP=inP, Qtot=totalQ)*100

## $T_{Ss}$

In [ ]:
seedDs=secondaryPipeDia #diameter to iterate
secondWorkL=10#long of second pipe section
servicePipeD=seedDs*0.8#contraction diameter to service pipe
servicePipeSects=numberOfMachines#the sections are equal to the number of machines
list_dpSecondPipes=[]#to store dropP of respective pressure
for i,j in zip(servicePipeSects, numberOfMachines):
    secondKAcces=[('Elbow, 90°, Long-radius (R/D = 1.5), All types',i),('Tee, Run, Screwed', i), ('Valve, Ball, Reduced trim, Beta = 0.9',1)]
    secondDropPi=checkdP_fromD(seedDia=seedDs, atmObj=atm, workLong=secondWorkL, kAcces=secondKAcces, secondPipeD=servicePipeD, secondPipeSects=j, inletP=inP, Qtot=totalQ)
    list_dpSecondPipes.append(secondDropPi)

In [ ]:
list_dpSecondPipes*100

# Units conversion to fill conventional data

In [ ]:
from fluids.units import *

## $Q_{partial}[CFM]$

In [ ]:
convenFlowPerMachines=flowPerMachines*((u.m**3)/u.s)
[round(i,3) for i in convenFlowPerMachines.to(other='(u.cubic_feet)/(u.min)')*utilizationCoeffs]

[20.023 <Unit('cubic_foot / minute')>,
 2.401 <Unit('cubic_foot / minute')>,
 3.761 <Unit('cubic_foot / minute')>,
 1.059 <Unit('cubic_foot / minute')>,
 0.989 <Unit('cubic_foot / minute')>]

## $Q_{tot}[CFM]$

In [ ]:
Qtot_CFM=round(np.sum(convenFlowPerMachines.to(other='(u.cubic_feet)/(u.min)')*utilizationCoeffs),3)
Qtot_CFM

28.234 <Unit('cubic_foot / minute')>

## $Q_{totFS}[CFM]$
Flow set at security factors

In [ ]:
np.sum(securityF*Qtot_CFM)+Qtot_CFM, securityF*Qtot_CFM

(43.7627 <Unit('cubic_foot / minute')>,
 array([4.2351, 2.8234, 2.8234, 5.6468]) <Unit('cubic_foot / minute')>)